In [8]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# Load the data
data = pd.read_parquet('./data_IDS2017/combined.parquet')

# Preprocessing
# Assuming your data requires scaling and splitting
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# Train-test split
X_train, X_test = train_test_split(data_scaled, test_size=0.2, random_state=42)

# Building the autoencoder
input_dim = X_train.shape[1]
encoding_dim = 14  # Number of neurons in the bottleneck layer, adjust as necessary

autoencoder = Sequential()
autoencoder.add(Dense(encoding_dim, activation='relu', input_shape=(input_dim,)))
autoencoder.add(Dense(input_dim, activation='sigmoid'))

autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Training the autoencoder
history = autoencoder.fit(X_train, X_train,
                          epochs=50,
                          batch_size=32,
                          shuffle=True,
                          validation_data=(X_test, X_test))

# Predictions
X_train_pred = autoencoder.predict(X_train)
X_test_pred = autoencoder.predict(X_test)

# Calculate reconstruction error
mse_train = np.mean(np.power(X_train - X_train_pred, 2), axis=1)
mse_test = np.mean(np.power(X_test - X_test_pred, 2), axis=1)

# Anomaly detection
threshold = np.percentile(mse_train, 95)  # You can adjust the threshold

# Identifying anomalies
anomalies = mse_test > threshold

print(f"Number of anomalies detected: {np.sum(anomalies)}")


ValueError: could not convert string to float: 'BENIGN'